In [6]:
import pandas as pd
import pandasql
import os


os.chdir("/Users/bencampbell/code_louisville/capstone/louisville-bike-accidents/")

DATA = "data/clean/bike_accidents.csv"
assert os.path.exists(DATA)

DATA = pd.read_csv(DATA)

SIGDATA = "data/preclean/signalized_intersections.csv"

SIG = pd.read_csv(SIGDATA)

In [12]:
pandasql.sqldf("SELECT roadway_name, main_street FROM DATA JOIN SIG on SIG.main_street == DATA.roadway_name", globals())

,roadway_name,main_street
0,BROWNSBORO,BROWNSBORO
1,BROWNSBORO,BROWNSBORO
2,BROWNSBORO,BROWNSBORO
3,BROWNSBORO,BROWNSBORO
4,BROWNSBORO,BROWNSBORO
...,...,...
11820,OUTER,OUTER
11821,OUTER,OUTER
11822,OUTER,OUTER
11823,OUTER,OUTER
